In [ ]:
from astropy.io import fits
from matplotlib.pyplot import figure, show, close
from scipy.optimize import curve_fit
import numpy as np

In [ ]:
# getting the MILES ccomparison spectrum for Arcturus
#

hdulist = fits.open('miles spectra/arcturus_miles.fits')
hdulist.info()
print()

hdr = hdulist[0].header
dat = hdulist[0].data

hdulist.close()

print(f"data shape: {dat.shape}")
arcturus_miles = dat[0,:]
print(hdr)

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(arcturus_miles,color="navy", label="Vega spectrum")
frame.set_title("Arcturus-like (K1) star (HD177463) spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

#============================================================

dat_arcturus = arcturus_miles[1000:4000] # choosing a relevant section of the data

x_range = np.arange(0, np.size(arcturus_miles[1000:4000]))
x_range_miles = x_range*0.9+3500+1000 # using the pixel -> Angstrom conversion values provided in the header, and accounting for cropping the data

def f_exp(x, a,b,z):
    """returns an exponential function of the form a*exp(b*x)"""
    return a*np.exp(b*x)+z

beta, pcov = curve_fit(f_exp, x_range_miles, dat_arcturus, p0=[2,-0.001,0.4])

print(beta)
exp_a,exp_b,exp_z = beta

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles,dat_arcturus,color="navy", label="Arcturus spectrum", s=0.2) # the section of the data that seems relevant
frame.plot(x_range_miles, f_exp(x_range_miles,exp_a,exp_b,exp_z), color="steelblue", label='Fitted Exponential Curve')
frame.set_title("Arcturus-like star spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

#============================================================

arcturus_miles_line=dat_arcturus-f_exp(x_range_miles,exp_a,exp_b,exp_z)

#============================================================

def f_superior_line(x, *params):
    """Returns a combination of multiple Gaussians plotted on a straight line"""
    num_gaussians = (len(params)-2) // 3
    y = np.zeros_like(x, dtype=float) #it was having some weird type issues
    for i in range(num_gaussians):
        a = params[3 * i]
        m = params[3 * i + 1]
        s = params[3 * i + 2]
        y += a * np.exp(-(x - m) ** 2 / (2.0 * s ** 2))
    # exponential part
    a,b = params[-2:]
    y += a*x+b
    return y

# using the parameters for the fit from our data, hence, redefining them in angstroms

gaussian_params_a=np.zeros(np.shape(gaussian_params))

p=0
for Peak in gaussian_params:
    a,m,s=Peak
    gaussian_params_a[p,0]=a
    gaussian_params_a[p,1]=m*a_in_pix+a_off
    gaussian_params_a[p,2]=s*a_in_pix
    p+=1

# had to fit them manually anyways due to the large discrepancy between our data and MILES

D1=np.array([-0.4,5300,1])
D8=np.array([-0.4,6000,5]) #Na I
D9 = np.array([-0.3,6300,5]) #?
D11 = np.array([-0.5,6650,5]) #Fe I

initial_gaussians = [D8,D11]
print(initial_gaussians)
initial_guess = [item for sublist in initial_gaussians for item in sublist] + [-0.00001,0]

beta, pcov = curve_fit(f_superior_line, x_range_miles, arcturus_miles_line, p0=initial_guess)
print("Parameters:", beta)

# extracting the fitted parameters
num_gaussians = len(initial_gaussians)
gaussian_params = beta[:num_gaussians * 3].reshape((num_gaussians, 3))
line_params = beta[num_gaussians * 3:]

# poor attempt at error analysis
err=np.sqrt(np.diag(pcov))
fwhm_err=2*err[2]*np.sqrt(2*np.log(2))

print("Gaussian parameters:", gaussian_params)
print("Line parameters:", line_params)

# plotting the final fit
fig = figure()
frame = fig.add_subplot(1, 1, 1)
frame.scatter(x_range_miles, arcturus_miles_line, color="orange", label="Arcturus spectrum", s=1)
frame.plot(x_range_miles, f_superior_line(x_range_miles, *beta), color="navy", label="Gaussian fit")
frame.set_title("Arcturus-like (K1) star (HD177463): peaks fitted")
frame.grid(alpha=0.2)
frame.legend()
show()
close(fig)

#============================================================


def f_line(x,a,b):
    """returns a straight line equation of the form ax+b"""
    return a*x+b

FWHM = np.array([2*np.abs(Peak[2])*np.sqrt(2*np.log(2)) for Peak in gaussian_params])
print(FWHM)

c=0
PeakDescriptions=["Na I", "H-alpha"]
PeakColors=["gold","cyan"]
for Peak in gaussian_params:
    print(f"{PeakDescriptions[c]} peak equivalent width: {FWHM[c]}")
    c+=1

print(f"Error in FWHMs: {fwhm_err}")

# 1.2056420239473526 4615.032805371546

fig=figure(figsize=(8,6))
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles,arcturus_miles_line, color="royalblue", label="Arcturus-like star spectrum",s=1)
frame.plot(x_range_miles,f_superior_line(x_range_miles, *beta),color="paleturquoise", label="Gaussian + linear fit")

p=0

PeakPositions = []
for Peak in gaussian_params:
    a,m,s=Peak
    PeakPositions.append(m)
    f=FWHM[p]
    fwhm_x=np.array((m-f/2,m+f/2))

    # getting the exponential fit data to know the y-coordinates of FWHM plot
    l_a,l_b=line_params
    z=f_line(m,l_a,l_b)

    fwhm_y=np.array((a/2+z,a/2+z))

    frame.plot(fwhm_x,fwhm_y,color=PeakColors[p], label=f"FWHM for {PeakDescriptions[p]} peak")
    p+=1

frame.set_title("Arcturus-like (K1) star (HD177463): Gaussian fit for the peaks and FWHMs")
frame.grid(alpha=0.2)
frame.set_ylabel("Intensity w.r.t the norm")
frame.set_xlabel("Wavelength [Angstroms]")
frame.legend(fontsize=7)
show()
close(fig)

#============================================================

hdulist = fits.open('miles spectra/vega_miles.fits')
hdulist.info()
print()

hdr = hdulist[0].header
dat = hdulist[0].data

hdulist.close()

print(f"data shape: {dat.shape}")
vega_miles = dat[0,:]

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(vega_miles,color="navy", label="Vega spectrum")
frame.set_title("Vega-like star spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

#============================================================

# defining a gaussian function for fitting the spectra

def f_gaussian(x, amp, mu, sigma, zero):
    """returns the value of a Gaussian function with amplitude amp centered at position mu with width sigma and 
    offset with respect to the x-axis off zero"""
    return amp * np.exp(-(x - mu) * (x - mu) / (2.0 * sigma * sigma)) + zero

#============================================================

dat_vega_0 = np.loadtxt("flatlined/Flatlined_Vega.txt", comments="#")


fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(dat_vega_0,color="navy", label="Vega spectrum")
frame.set_title("vega_wl")
frame.grid(alpha=0.2)
show()
close(fig)

hdulist.close()


#============================================================

# Making a Gaussian fit for the 1st absorption peak, plotting it

beta, pcov1 = curve_fit(f_gaussian, np.arange(1, dat_vega_0.size+1), dat_vega_0, p0=(-0.6, 1800, 40, 1))
#amp, mu, sigma, zero

print(beta)

a1,m1,s1,z1=beta
x_range = np.arange(1, dat_vega_0.size+1)

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range,dat_vega_0,color="royalblue", label="Vega spectrum",s=1)
frame.plot(f_gaussian(x_range,a1,m1,s1,z1),color="paleturquoise", label="fit for the 1st peak")
frame.set_title("Vega: 1st peak fitted")
frame.grid(alpha=0.2)
frame.legend()
show()
close(fig)

#============================================================

# I will take the offset fit for all three peak fits and take the mean, to obtain the average value
# or the weighted average, i'll see which one yields more accurate data

beta, pcov2 = curve_fit(f_gaussian, x_range, dat_vega_0, p0=(-0.2, 400, 20, 1.02))
a2,m2,s2,z2=beta

# separate code for the 3rd peak because it's being quirky
# my interpretation: H-alpha (~6600 A), H-beta (~4850 A), O2 band (~6950)

x_range_3rdpeak = np.arange(100,200)

beta, pcov3 = curve_fit(f_gaussian, x_range_3rdpeak, dat_vega_0[100:200], p0=(-0.4, 150, 20, 1.05))

a3,m3,s3,z3=beta

z = np.median([z1,z2,z3])

f_composed = f_gaussian(x_range,a1,m1,s1,z)+f_gaussian(x_range,a2,m2,s2,0)+f_gaussian(x_range,a3,m3,s3,0)
print(f_composed)
fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range,dat_vega_0,color="royalblue", label="Vega spectrum",s=1)
frame.plot(f_composed,color="paleturquoise", label="Gaussian fit")
frame.set_title("Vega: peaks fitted")
frame.grid(alpha=0.2)
frame.legend()
show()
close(fig)

#============================================================

def fwhm_calculator(sigma,pcov):
    """Calculates the equivalent width from a given sigma value and its error using the pcov matrix"""
    fwhm=2*sigma*np.sqrt(2*np.log(2))
    err=np.sqrt(np.diag(pcov))
    fwhm_err=2*err[2]*np.sqrt(2*np.log(2))
    return fwhm,fwhm_err

fwhm1,fwhm1_err=fwhm_calculator(s1,pcov1)
fwhm2,fwhm2_err=fwhm_calculator(s2,pcov2)
fwhm3,fwhm3_err=fwhm_calculator(s3,pcov3)

print(f"H-beta peak width: {fwhm1}, error: {fwhm1_err}")
print(f"H-alpha peak width: {fwhm2}, error: {fwhm2_err}")
print(f"O2 band width: {fwhm3}, error: {fwhm3_err}")

#============================================================

# defining points for plotting the FWHMs

fwhm1_x=np.array((m1-fwhm1/2,m1+fwhm1/2))
fwhm1_y=np.array((a1/2+z,a1/2+z))

fwhm2_x=np.array((m2-fwhm2/2,m2+fwhm2/2))
fwhm2_y=np.array((a2/2+z,a2/2+z))

fwhm3_x=np.array((m3-fwhm3/2,m3+fwhm3/2))
fwhm3_y=np.array((a3/2+z,a3/2+z))

#============================================================

# Converting the values from pixels to Angstroms using values from wavelength calibration

a_in_pix=-1.217
a_off = 7060

def pix_to_A (a):
    """Function to simplify the conversion of values/numpy array from values in pixels to Angstroms"""
    return a*a_in_pix+a_off

x_range_a = pix_to_A(x_range)
x_range_a = a_in_pix*x_range + a_off
print(a_in_pix,a_off)

print(f"H-alpha peak width: {a_in_pix*fwhm1} Angstroms, error: {a_in_pix*fwhm1_err}")
print(f"H-beta peak width: {a_in_pix*fwhm2} Angstroms, error: {a_in_pix*fwhm2_err}")
print(f"O2 band width: {a_in_pix*fwhm3} Angstroms, error: {a_in_pix*fwhm3_err}")

m1_a = pix_to_A(m1)
m2_a = pix_to_A(m2)
m3_a = pix_to_A(m3)

print(f"H-beta peak position: {m1_a} Angstroms")
print(f"H-alpha peak position: {m2_a} Angstroms")
print(f"O2 band peak position: {m3_a:.2f} Angstroms")



fig=figure(figsize=(8,6))
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_a,dat_vega_0,color="orange", label="Vega spectrum",s=1)
frame.plot(x_range_a,f_composed, label="Gaussian fit")
frame.plot(fwhm1_x*a_in_pix+a_off,fwhm1_y,color="darkgoldenrod", label="FWHM for H-beta peak")
frame.plot(fwhm2_x*a_in_pix+a_off,fwhm2_y,color="cyan", label="FWHM for H-alpha peak")
frame.plot(fwhm3_x*a_in_pix+a_off,fwhm3_y,color="green", label="FWHM for O2 band")
frame.set_title("Vega: Gaussian fit for the peaks and FWHMs")
frame.grid(alpha=0.2)
frame.set_ylabel("Intensity w.r.t the norm")
frame.set_xlabel("Wavelength [Angstroms]")
frame.legend(fontsize=8.5)
show()
close(fig)

# C R I S P

#wavelength calibration from Stefan: -1.217x + 7060

#============================================================

# the fight with MILES continues

print(vega_miles[1000:4000])

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(vega_miles[1000:4000],color="navy", label="Vega spectrum") # the section of the data that seems relevant
frame.set_title("Vega-like star spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

print(vega_miles[1000:4000])

#============================================================

# defining the normalization function using the normal statistics formula
def normalize_spectrum(wavelengths, intensities):
    min_intensity = np.min(intensities)
    max_intensity = np.max(intensities)
    normalized_intensities = (intensities - min_intensity) / (max_intensity - min_intensity)
    return normalized_intensities

#defining the normalization function using some quirky thing from PROGNUM material
def normalize_quirky(wavelengths, intensities):
    normalised_spectrum = intensities/np.median(intensities)
    return normalised_spectrum


x_range_miles=np.arange(1000, 4000)

normalized_vega_miles = normalize_spectrum(x_range_miles, vega_miles[1000:4000])

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(normalized_vega_miles,color="yellow", label="Vega spectrum")
frame.set_title("normalised-1 from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

normalized_vega_miles2 = normalize_quirky(x_range_miles, vega_miles[1000:4000])

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.plot(normalized_vega_miles2,color="yellow", label="Vega spectrum")
frame.set_title("normalised-2 from MILES")
frame.grid(alpha=0.2)
show()
close(fig)


#============================================================

def f_exp(x, a,b,z):
    """returns an exponential function of the form a*exp(b*x)"""
    return a*np.exp(b*x)+z

# fitting exponential curve to the miles Vega

beta, pcov = curve_fit(f_exp, x_range_miles, vega_miles[1000:4000], p0=[2,-0.001,0.4])

print(beta)
exp_a,exp_b,exp_z = beta

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles,vega_miles[1000:4000],color="navy", label="Vega spectrum", s=0.2) # the section of the data that seems relevant
frame.plot(x_range_miles, f_exp(x_range_miles,exp_a,exp_b,exp_z), color="steelblue", label='Fitted Exponential Curve')
frame.set_title("Vega-like star spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

#============================================================

# subtracting the exponential fit from the MILES data to obtain a straight line
vega_miles_line=vega_miles[1000:4000]-f_exp(x_range_miles,exp_a,exp_b,exp_z)

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles,vega_miles_line,color="navy", label="Vega spectrum", s=0.2) # the section of the data that seems relevant
frame.set_title("Vega-like star spectrum from MILES")
frame.grid(alpha=0.2)
show()
close(fig)

#============================================================

beta, pcov1 = curve_fit(f_gaussian, x_range_miles, vega_miles_line, p0=(-0.85,1500,100,0))
a1,m1,s1,z1=beta
beta, pcov2 = curve_fit(f_gaussian, x_range_miles, vega_miles_line, p0=(-0.3,3400,100,0))
a2,m2,s2,z2=beta

z=np.mean([z1,z2])

f_composed = f_gaussian(x_range_miles,a1,m1,s1,z)+f_gaussian(x_range_miles,a2,m2,s2,0)

fig=figure()
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles,vega_miles_line,color="navy", label="Vega spectrum (MILES)",s=1)
frame.plot(x_range_miles,f_composed,color="steelblue", label="Gaussian fit")
frame.set_title("Vega: peaks fitted")
frame.grid(alpha=0.2)
frame.legend()
show()
close(fig)

#============================================================

a_in_pix,a_off = 0.9, 3500 # information from the MILES data header file


fwhm1,fwhm1_err=fwhm_calculator(s1,pcov1)
fwhm2,fwhm2_err=fwhm_calculator(s2,pcov2)

fwhm1_x=np.array((m1-fwhm1/2,m1+fwhm1/2))
fwhm1_y=np.array((a1/2+z,a1/2+z))

fwhm2_x=np.array((m2-fwhm2/2,m2+fwhm2/2))
fwhm2_y=np.array((a2/2+z,a2/2+z))

m1_a = a_in_pix*m1+a_off
m2_a = a_in_pix*m2+a_off

print(f"H-beta peak width: {fwhm1}, error: {fwhm1_err}")
print(f"H-alpha peak width: {fwhm2}, error: {fwhm2_err}")
print(f"H-beta peak position: {m1_a} Angstroms")
print(f"H-alpha peak position: {m2_a} Angstroms")

x_range_miles_a = x_range_miles*a_in_pix+a_off

print(f"H-beta peak width: {a_in_pix*fwhm1} Angstroms, error: {a_in_pix*fwhm1_err}")
print(f"H-alpha peak width: {a_in_pix*fwhm2} Angstroms, error: {a_in_pix*fwhm2_err}")

fig=figure(figsize=(8,6))
frame=fig.add_subplot(1,1,1)
frame.scatter(x_range_miles_a,vega_miles_line,color="orange", label="Vega spectrum [MILES]",s=1)
frame.plot(x_range_miles_a,f_composed, label="Gaussian fit")
frame.plot(fwhm1_x*a_in_pix+a_off,fwhm1_y,color="darkgoldenrod", label="FWHM for H-beta line")
frame.plot(fwhm2_x*a_in_pix+a_off,fwhm2_y,color="cyan", label="FWHM for H-alpha line")
frame.set_title("Vega-like (A0V) star (HD174567): Gaussian fit for the peaks and FWHMs")
frame.grid(alpha=0.2)
frame.set_ylabel("Intensity w.r.t the norm")
frame.set_xlabel("Wavelength [Angstroms]")
frame.legend(fontsize=8.5)
show()
close(fig)

